In [1]:
pip install gdown

In [2]:
import gdown
url = 'https://drive.google.com/uc?id=1UOJcE-YEfPqhF65Gc1wrZx9N16kLLmf6'
output = 'train_test_zip'
gdown.download(url, output)

In [3]:
!unzip train_test_zip

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import random
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import shutil
import scipy

In [5]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)


dataset_dir  = r'kaggle/working'
training_dir = os.path.join(dataset_dir,'train')
val_dir      = os.path.join(dataset_dir,'val')
labels       = {
    0:'Apple',     
    1:'Blueberry', 
    2:'Cherry',    
    3:'Corn',      
    4:'Grape',     
    5:'Orange',    
    6:'Peach',     
    7:'Pepper',    
    8:'Potato',    
    9:'Raspberry', 
    10:'Soybean',   
    11:'Squash',     
    12:'Strawberry',
    13:'Tomato'
}

In [6]:
# Creating an instance of ImageDataGenerator without Data Augmentation
noaug_train_data_gen = ImageDataGenerator(preprocessing_function=tfk.applications.resnet50.preprocess_input)
valid_data_gen       = ImageDataGenerator(preprocessing_function=tfk.applications.resnet50.preprocess_input)
test_data_gen        = ImageDataGenerator(preprocessing_function=tfk.applications.resnet50.preprocess_input)

os.chdir('/')

noaug_train_gen = noaug_train_data_gen.flow_from_directory(directory=training_dir,
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=64,
                                                           shuffle=True,
                                                           seed=seed)

valid_gen = valid_data_gen.flow_from_directory(directory=val_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=64,
                                               shuffle=False, # no need to shuffle validation set
                                               seed=seed)

In [7]:
supernet = tfk.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
)

In [8]:
# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(256,256,3))
x = supernet(inputs)
x1 = tfkl.GlobalAveragePooling2D(name="gl_avg_pool")(x)
x2 = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
x3 = tfkl.Dropout(0.3, seed=seed)(x2)
x4 = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x3)
x5 = tfkl.Dropout(0.3, seed=seed)(x4)
outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x5)
    
# Connect input and output through the Model class
tl_model = tfk.models.Model(inputs = inputs, outputs = outputs, name = 'model')


# Compile the model
tl_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate = 1e-4), metrics=[tfk.metrics.CategoricalAccuracy(), tfk.metrics.AUC()])

In [11]:
supernet.trainable = True
for i, layer in enumerate(tl_model.get_layer('resnet50').layers[:-32]):
    layer.trainable=False
for i, layer in enumerate(tl_model.get_layer('resnet50').layers):
    print(i, layer.name, layer.trainable)

In [13]:
# Train the model
os.chdir('/')
tl_history = tl_model.fit(
    x = noaug_train_gen,
    epochs = 150,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True),
                tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.1, patience=3, verbose=1,
                mode='min', min_delta=0.000001)]
).history

In [14]:
os.chdir('/kaggle/working')
tl_model.save('rn_model_with_data_aug')

In [15]:
shutil.make_archive("rn", "zip", "rn_model_with_data_aug")
from IPython.display import FileLink
FileLink(r'rn.zip')